In [7]:
import os
import cv2
from imgaug import augmenters as iaa
import numpy as np
from glob import glob
from sklearn.model_selection import train_test_split

In [2]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.models import load_model

In [4]:
def extract_frames_from_videos(video_dir, output_dir, frames_per_video=30):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    video_files = [f for f in os.listdir(video_dir) if f.endswith('.mp4')]
    
    for video_file in video_files:
        video_path = os.path.join(video_dir, video_file)
        video_capture = cv2.VideoCapture(video_path)
        frame_count = int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))
        
        # Check if there are frames in the video
        if frame_count == 0:
            print(f"Skipping {video_file} due to zero frames.")
            continue
        
        success, image = video_capture.read()
        count = 0
        step = max(1, frame_count // frames_per_video)  # Ensure step is at least 1
        
        while success:
            if count % step == 0:
                frame_filename = os.path.join(output_dir, f"{video_file}_frame{count}.jpg")
                cv2.imwrite(frame_filename, image)
            success, image = video_capture.read()
            count += 1

        video_capture.release()  # Release the video capture object

# Define paths
dataset_dir = 'dataset'
accident_dir = os.path.join(dataset_dir, 'Accident')
normal_dir = os.path.join(dataset_dir, 'Normal')
frames_accident_dir = os.path.join(dataset_dir, 'Frames_Accident')
frames_normal_dir = os.path.join(dataset_dir, 'Frames_Normal')

# Extract frames
extract_frames_from_videos(accident_dir, frames_accident_dir)
extract_frames_from_videos(normal_dir, frames_normal_dir)


In [9]:
from imgaug import augmenters as iaa
import numpy as np
from glob import glob
import cv2

def augment_images(image_dir, output_dir, augment_count=3):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Define augmentations
    seq = iaa.Sequential([
        iaa.Fliplr(0.5),  # horizontal flip
        iaa.Affine(rotate=(-10, 10)),  # rotation
        iaa.Multiply((0.8, 1.2)),  # change brightness
        iaa.GaussianBlur(sigma=(0.0, 3.0))  # blur
    ])
    
    image_files = glob(os.path.join(image_dir, '*.jpg'))
    
    for image_file in image_files:
        # Read image
        image = cv2.imread(image_file)
        
        # Check if image was read correctly
        if image is None:
            print(f"Warning: Could not read image {image_file}. Skipping.")
            continue
        
        # Convert to RGB (if needed)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Ensure image shape is valid
        if len(image.shape) != 3 or image.shape[2] != 3:
            print(f"Warning: Image {image_file} has unexpected shape {image.shape}. Skipping.")
            continue
        
        # Perform augmentation
        images_aug = [image] + [seq.augment_image(image) for _ in range(augment_count)]
        
        for i, img_aug in enumerate(images_aug):
            # Convert back to BGR for saving
            img_aug_bgr = cv2.cvtColor(img_aug, cv2.COLOR_RGB2BGR)
            
            aug_filename = os.path.join(output_dir, f"{os.path.basename(image_file).split('.')[0]}_aug{i}.jpg")
            cv2.imwrite(aug_filename, img_aug_bgr)

# Augment frames from normal videos
augmented_normal_dir = os.path.join(dataset_dir, 'Augmented_Frames_Normal')
augment_images(frames_normal_dir, augmented_normal_dir)


In [12]:
import os
import cv2
import numpy as np
from keras.applications.mobilenet import MobileNet, preprocess_input

def extract_features_from_frames(directory, model, input_size=(224, 224)):
    features = []
    image_files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.jpg') or f.endswith('.png')]

    for image_file in image_files:
        image = cv2.imread(image_file)
        if image is None:
            print(f"Warning: Unable to read image {image_file}. Skipping.")
            continue

        try:
            image = cv2.resize(image, input_size)
            image = preprocess_input(image)
            image = np.expand_dims(image, axis=0)
            feature = model.predict(image)
            features.append(feature)
        except Exception as e:
            print(f"Error processing image {image_file}: {e}")
            continue

    return np.array(features)

# Example usage
frames_accident_dir = 'dataset/Frames_Accident'
augmented_normal_dir = 'dataset/Augmented_Frames_Normal'

mobilenet_model = MobileNet(weights='imagenet', include_top=False, pooling='avg')

# Extract features from accident frames
accident_features = extract_features_from_frames(frames_accident_dir, mobilenet_model)

# Extract features from augmented normal frames
normal_features = extract_features_from_frames(augmented_normal_dir, mobilenet_model)


1/1 [==============================] - 0s 31ms/step


In [1]:
# Combine features and labels
all_features = accident_features + normal_features
all_labels = [1] * len(accident_features) + [0] * len(normal_features)

# Split into train, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(all_features, all_labels, test_size=0.4, stratify=all_labels, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42)

NameError: name 'accident_features' is not defined

In [ ]:
def create_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(128, input_shape=input_shape, return_sequences=True))
    model.add(Dropout(0.5))
    model.add(LSTM(64))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Prepare data for LSTM
X_train_lstm = np.array([x[0] for x in X_train])
X_val_lstm = np.array([x[0] for x in X_val])
X_test_lstm = np.array([x[0] for x in X_test])

# Create and train model
lstm_model = create_lstm_model(X_train_lstm.shape[1:])
lstm_model.fit(X_train_lstm, np.array(y_train), validation_data=(X_val_lstm, np.array(y_val)), epochs=10, batch_size=32)

# Save the model
lstm_model.save('mobilenet_lstm_model.h5')